<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

df = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


In [3]:
df['text'][0]

'BEWARE!!! FAKE, FAKE, FAKE....We also own a small business in Los Alamitos, CA and received what looked like a legitimate bill for $70 with an account number and all.  I called the phone number listed (866) 273-7934.  The wait time on hold said 20 minutes and to leave a message.  I could not get a live person on the phone no matter what number I selected.  I left a very FIRM message that I would be contacting the BBB and my attorney regarding their company trying to scam businesses. This has to be illegal!!!!!'

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [4]:
import re
import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt

In [5]:
nlp = spacy.load("en_core_web_lg")

In [11]:
def tokenize(doc):
    '''tokenizer, lemmatizer, stopword remover'''
    tokenizer = Tokenizer(nlp.vocab)
    tokens_list = [token.lemma_ for token in tokenizer(doc) if (token.is_stop == False) & (token.is_punct == False) & (token.is_space == False)]
    clean_doc = ''
    for token in tokens_list:
        clean_doc += token + ' '
    return tokens_list

In [12]:
tokenize("Today is gonna be the day that they throw it all back to you...")

['Today', 'gonna', 'day', 'throw', 'you...']

In [13]:
df['tokens'] = df['text'].apply(tokenize)

In [14]:
df['tokens'][0]

['BEWARE!!!',
 'FAKE,',
 'FAKE,',
 'FAKE....We',
 'small',
 'business',
 'Los',
 'Alamitos,',
 'receive',
 'look',
 'like',
 'legitimate',
 'bill',
 '$70',
 'account',
 'numb',
 'all.',
 'call',
 'phone',
 'numb',
 'list',
 '(866)',
 '273-7934.',
 'wait',
 'time',
 'hold',
 'say',
 '20',
 'minute',
 'leave',
 'message.',
 'live',
 'person',
 'phone',
 'matt',
 'numb',
 'selected.',
 'leave',
 'FIRM',
 'message',
 'contact',
 'BBB',
 'attorney',
 'company',
 'try',
 'scam',
 'businesses.',
 'illegal!!!!!']

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [15]:
counter = TfidfVectorizer(max_df = .97,
                          min_df = 3,
                          stop_words='english',
                          ngram_range= (1,2),
                          tokenizer=tokenize)

In [16]:
dtm = counter.fit_transform(df['text'])

In [17]:
dtm = pd.DataFrame(dtm.todense(), columns=counter.get_feature_names())

In [18]:
dtm.head()

,"""a","""all","""are","""as","""be","""best","""big","""build","""but","""can",...,à,à ce,à essayer,à la,ça,équipe,était,été,être,‍
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
from sklearn.neighbors import NearestNeighbors

In [20]:
nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [21]:
fake_review = ['''BEWARE!!! FAKE, FAKE, FAKE....We also own a small business in Los Alamitos, CA and received what looked like a legitimate bill for $70 with an account number and all.  I called the phone number listed (866) 273-7934.  The wait time on hold said 20 minutes and to leave a message.  I could not get a live person on the phone no matter what number I selected.  I left a very FIRM message that I would be contacting the BBB and my attorney regarding their company trying to scam businesses. This has to be illegal!!!!!''']
fake_review_counter = counter.transform(fake_review)
arrays = nn.kneighbors(fake_review_counter.todense())

In [22]:
for i in arrays[1][0].tolist():
    print(df['text'][i][:100])

BEWARE!!! FAKE, FAKE, FAKE....We also own a small business in Los Alamitos, CA and received what loo
旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした。
質問にも丁寧に答えてくれましたし、日本人の方も日本語が話せる方も居て、とても綺麗で居心地のいいお店でした。 
ネイルはちは
天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用心製作，包含擺盤精緻、佐料衛生，夏日想開胃，這是一個不錯的選擇，服務人員也很敬業，以後會常常來
Hi, 
Update: I went to the location and spoke to the people around, they think there is something fi
Haven't even gotten the chance to hear a quote or anything from them because they never pick up the 
Do not use PROFESSIONAL TIMESHARE SERVICES! They do not help you! This is a scam to get $2495.00 of 
Well, from the outside it looks like a pretty cool store. My aunt and uncle ordered some figure/stat
if could leave a 0 star i would i was on hold for 2 hours and there system just hung up on me everyt
so only a short time after my first visit, i dropped my phone and cracked the hell out of my screen.
I went here today, and it was one of the best experiences with a phone company I have EVER had. I 

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [28]:
X = df['text']
y = df['stars']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, random_state=42, shuffle=True) 

C:\Users\tally\.virtualenvs\Repositories-J01Zxu1S\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [29]:
pipeline = Pipeline([('counter', counter),
                     ('rf', RandomForestClassifier())])

In [30]:
parameters = {
    'counter__max_df': [i/100 for i in range(75, 100)],
    'counter__min_df': range(0, 10),
    'counter__ngram_range': [(1,2), (1,1), (1, 3)],
    'rf__max_depth':(5,10,15,20),
    'rf__min_samples_split': range(2, 10),
    'rf__min_samples_leaf': range(1,50)
}

grid_search = RandomizedSearchCV(pipeline,parameters, cv=5, n_iter=2, n_jobs=1, verbose=5)
search = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] rf__min_samples_split=7, rf__min_samples_leaf=28, rf__max_depth=15, counter__ngram_range=(1, 1), counter__min_df=4, counter__max_df=0.86 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\tally\.virtualenvs\Repositories-J01Zxu1S\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[CV]  rf__min_samples_split=7, rf__min_samples_leaf=28, rf__max_depth=15, counter__ngram_range=(1, 1), counter__min_df=4, counter__max_df=0.86, score=0.4701492537313433, total=   1.7s
[CV] rf__min_samples_split=7, rf__min_samples_leaf=28, rf__max_depth=15, counter__ngram_range=(1, 1), counter__min_df=4, counter__max_df=0.86 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.0s remaining:    0.0s
C:\Users\tally\.virtualenvs\Repositories-J01Zxu1S\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[CV]  rf__min_samples_split=7, rf__min_samples_leaf=28, rf__max_depth=15, counter__ngram_range=(1, 1), counter__min_df=4, counter__max_df=0.86, score=0.4613466334164589, total=   1.7s
[CV] rf__min_samples_split=7, rf__min_samples_leaf=28, rf__max_depth=15, counter__ngram_range=(1, 1), counter__min_df=4, counter__max_df=0.86 


...

C:\Users\tally\.virtualenvs\Repositories-J01Zxu1S\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[CV]  rf__min_samples_split=7, rf__min_samples_leaf=11, rf__max_depth=10, counter__ngram_range=(1, 2), counter__min_df=7, counter__max_df=0.89, score=0.47619047619047616, total=   2.0s
[CV] rf__min_samples_split=7, rf__min_samples_leaf=11, rf__max_depth=10, counter__ngram_range=(1, 2), counter__min_df=7, counter__max_df=0.89 


C:\Users\tally\.virtualenvs\Repositories-J01Zxu1S\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[CV]  rf__min_samples_split=7, rf__min_samples_leaf=11, rf__max_depth=10, counter__ngram_range=(1, 2), counter__min_df=7, counter__max_df=0.89, score=0.47858942065491183, total=   1.9s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   34.1s finished
C:\Users\tally\.virtualenvs\Repositories-J01Zxu1S\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [40]:
import numpy as np
np.unique(search.predict(X_test), return_counts=True)

(array([1, 4, 5], dtype=int64), array([ 166,   83, 7751], dtype=int64))

In [35]:
X_test.dtype

dtype('O')

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [ ]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

In [ ]:
df['tokens'] = [tokenize(text) for text in df['text']]
df['tokens'].head()

Learn the vocubalary of the yelp data:

In [ ]:
id2word = Dictionary(df['tokens'])

Create a bag of words representation of the entire corpus

In [ ]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

Your LDA model should be ready for estimation: 

In [ ]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=3,
                   num_topics = 5 # You can change this parameter
                  )

In [ ]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

Create 1-2 visualizations of the results

The top right corner, or Topic 1, seems to represent the most positive reviews.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)